Import the necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

Load the pre-processed data

In [2]:
# Load the pre-processed data
features_normalized = np.load('Pre_Processed_Data/features_normalized.npy')
labels = np.load('Pre_Processed_Data/labels.npy')

# Convert the data to PyTorch tensors
features_normalized = torch.tensor(features_normalized).float()
labels = torch.tensor(labels).long()


Define a custom dataset and create a DataLoader

In [3]:
# Define a custom dataset
class ParkinsonsDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create a DataLoader
dataset = ParkinsonsDataset(features_normalized, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Define the neural network architecture

In [4]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(features_normalized.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the neural network
net = Net()

Define the loss function and the optimizer

In [5]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Train the neural network

In [6]:
# Train the neural network
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch: {epoch} Loss: {running_loss / len(dataloader)}')

Epoch: 0 Loss: 0.6218272021838597
Epoch: 1 Loss: 0.550872415304184
Epoch: 2 Loss: 0.5397496223449707
Epoch: 3 Loss: 0.5125875515597207
Epoch: 4 Loss: 0.4295012003609112
Epoch: 5 Loss: 0.4203262392963682
Epoch: 6 Loss: 0.4359692462853023
Epoch: 7 Loss: 0.4298115244933537
Epoch: 8 Loss: 0.4244547826903207
Epoch: 9 Loss: 0.4579229269708906
Epoch: 10 Loss: 0.3723395892551967
Epoch: 11 Loss: 0.359037663255419
Epoch: 12 Loss: 0.3802281107221331
Epoch: 13 Loss: 0.3361007102898189
Epoch: 14 Loss: 0.31338574843747274
Epoch: 15 Loss: 0.3629936490740095
Epoch: 16 Loss: 0.30719466294561115
Epoch: 17 Loss: 0.36636551150253843
Epoch: 18 Loss: 0.3873536842209952
Epoch: 19 Loss: 0.3134868230138506
Epoch: 20 Loss: 0.3539097649710519
Epoch: 21 Loss: 0.29480829409190584
Epoch: 22 Loss: 0.3107388360159738
Epoch: 23 Loss: 0.26916430411594255
Epoch: 24 Loss: 0.33015501712049755
Epoch: 25 Loss: 0.3118360298020499
Epoch: 26 Loss: 0.2687418503420694
Epoch: 27 Loss: 0.2699644608157022
Epoch: 28 Loss: 0.24865084

Make predictions using the trained neural network

In [7]:
# Make predictions on multiple samples
predicted_classes = []
for i in range(features_normalized.shape[0]):
    sample = features_normalized[i]
    output = net(sample)
    _, predicted_class = torch.max(output, 0)
    predicted_classes.append(predicted_class.item())

print(f'Predicted classes: {predicted_classes}')

Predicted classes: [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
